# Publishing data with OME Zarr

Driven by the need to quickly publish a recent COVID dataset in the IDR, the OME team has accelerated development of an archival version of bioimaging data in a cloud-enabled format for easier download and re-analysis. A number of images from the Image Data Resource (IDR) have been converted using the bioformats2raw tool into the Zarr format and then transferred to EBI’s S3 servers for public download. By working from a common representation, these files can be scalably accessed over common web protocols.

## Workflows

<img src="zarr-diagram.png" style="height:300px" />

### bioformats2raw

In [1]:
BFVERSION="0.2.1"

In [2]:
%%bash -s "$BFVERSION"
BFREPO=https://github.com/glencoesoftware/bioformats2raw/releases/download
BF2RAW=bioformats2raw-${1}
test -e ${BF2RAW}.zip || wget ${BFREPO}/v${BF2RAW}/${BF2RAW}.zip
test -e ${BF2RAW} || unzip ${BF2RAW}.zip

In [3]:
!bioformats2raw-{BFVERSION}/bin/bioformats2raw

Missing required parameters: <inputPath>, <outputPath>
Usage: <main class> [--debug] [--extra-readers[=<extraReaders>[,
                    <extraReaders>...]]]...
                    [--additional-scale-format-string-args=<additionalScaleForma
                    tStringArgsCsv>] [-c=<compressionType>]
                    [--compression-parameter=<compressionParameter>]
                    [--dimension-order=<dimensionOrder>]
                    [--file_type=<fileType>] [-h=<tileHeight>]
                    [--max_cached_tiles=<maxCachedTiles>]
                    [--max_workers=<maxWorkers>] [--pyramid-name=<pyramidName>]
                    [-r=<pyramidResolutions>]
                    [--scale-format-string=<scaleFormatString>]
                    [-w=<tileWidth>] <inputPath> <outputPath>
      <inputPath>    file to convert
      <outputPath>   path to the output pyramid directory
      --additional-scale-format-string-args=<additionalScaleFormatStringArgsCsv>
                    

In [ ]:
!bioformats2raw-{BFVERSION}/bin/bioformats2raw --file_type=zarr --dimension-order=XYZCT a.fake /tmp/output

2020-05-19 23:14:45,352 [main] INFO  c.g.bioformats2raw.Converter - Output will be incompatible with raw2ometiff (pyramidName: data.zarr, scaleFormatString: %d/%d)
2020-05-19 23:14:45,692 [main] INFO  loci.formats.ImageReader - FakeReader initializing a.fake
2020-05-19 23:14:45,830 [main] INFO  c.g.bioformats2raw.Converter - Using 2 pyramid resolutions
2020-05-19 23:14:45,830 [main] INFO  c.g.bioformats2raw.Converter - Preparing to write pyramid sizeX 512 (tileWidth: 1024) sizeY 512 (tileWidth: 1024) imageCount 1
2020-05-19 23:14:46,080 [main] WARN  c.g.bioformats2raw.Converter - Reducing active tileWidth to 512
2020-05-19 23:14:46,081 [main] WARN  c.g.bioformats2raw.Converter - Reducing active tileHeight to 512
2020-05-19 23:14:46,094 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 0, 0, 0] to /0/0
2020-05-19 23:14:46,104 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - tile read complete 1/1
2020-05-19 23:14:46,104 [pool-1-thread-1] INF

In [2]:
!java --version

openjdk 11.0.7 2020-04-14
OpenJDK Runtime Environment (build 11.0.7+10)
OpenJDK 64-Bit Server VM (build 11.0.7+10, mixed mode)


### omero-ms-zarr

In [4]:
MSVERSION="0.1.1"

In [8]:
%%bash -s "$MSVERSION"
MSREPO=https://github.com/ome/omero-ms-zarr/archive
MSZARR=v${1}
test -e ${MSZARR}.zip || wget ${MSREPO}/${MSZARR}.zip
test -e omero-ms-zarr-${1} || unzip ${MSZARR}.zip

In [10]:
%%script --bg bash
cd omero-ms-zarr-0.1.1
cat <<EOF > cfg
omero.data.dir=/tmp/ome1
omero.db.name=ome1
omero.db.user=postgres
omero.ms.zarr.net.path.image=/idr/zarr/v0.1/{image}.zarr/
EOF
gradle run --args=cfg

In [16]:
import requests
requests.get("http://localhost:8080/idr/zarr/v0.1/1.zarr")

<Response [404]>

In [ ]:
import ome_zarr

### Load the ilastik projects linked to the dataset

In [21]:
def load_model(dataset_id):
    path = tempfile.mkdtemp()
    if not os.path.exists(path):
        os.makedirs(path)
    dataset = conn.getObject("Dataset", dataset_id)
    # Go through all the annotations on the Dataset
    options = []
    for ann in dataset.listAnnotations():
        if isinstance(ann, omero.gateway.FileAnnotationWrapper):
            name = ann.getFile().getName()
            # Select the ilatisk project TODO: use namespace
            if name.endswith(".ilp"):
                file_path = os.path.join(path, name)
                options.append((name, file_path))
                with open(str(file_path), 'wb') as f:
                    for chunk in ann.getFileInChunks():
                        f.write(chunk)
    return widgets.Dropdown(options=options, disabled=False)

### Helper function: load an Image as 5D-numpy array: order TZYXC

In [22]:
def load_numpy_array(image):
    pixels = image.getPrimaryPixels()
    size_z = image.getSizeZ()
    size_c = image.getSizeC()
    size_t = image.getSizeT()
    size_y = image.getSizeY()
    size_x = image.getSizeX()
    z, t, c = 0, 0, 0  # first plane of the image

    zct_list = []
    for t in range(size_t):
        for z in range(size_z):  # get the Z-stack
            for c in range(size_c):  # all channels
                zct_list.append((z, c, t))

    values = []
    # Load all the planes as YX numpy array
    planes = pixels.getPlanes(zct_list)
    j = 0
    k = 0
    tmp_c = []
    tmp_z = []
    s = "z:%s t:%s c:%s y:%s x:%s" % (size_z, size_t, size_c, size_y, size_x)
    print(s)
    # axis tzyxc
    print("Downloading image %s" % image.getName())
    for i, p in enumerate(planes):
        if k < size_z:
            if j < size_c:
                tmp_c.append(p)
                j = j + 1
            if j == size_c:
                # use dstack to have c at the end
                tmp_z.append(numpy.dstack(tmp_c))
                tmp_c = []
                j = 0
                k = k + 1
        if k == size_z:  # done with the stack
            values.append(numpy.stack(tmp_z))
            tmp_z = []
            k = 0

    return numpy.stack(values)

In [23]:
def plane_gen():
    """
    Set up a generator of 2D numpy arrays.

    The createImage method below expects planes in the order specified here
    (for z.. for c.. for t..)

    """

    size_z = data.shape[0]-1
    for z in range(data.shape[0]):  # all Z sections data.shape[0]
        print('z: %s/%s' % (z, size_z))
        for c in range(data.shape[1]):  # all channels
            for t in range(data.shape[2]):  # all time-points
                yield data[z][c][t]

### Select the ilastik project to use.

In [23]:
model_selection = load_model(dataset_id)
display(model_selection)

### Load each image as an 5D-numpy array and analyze.
Save the probabilities as an OMERO image

In [25]:
# Load the model linked to the dataset
model_file = model_selection.value

images = conn.getObjects('Image', opts={'dataset': dataset_id})

# Create a new dataset where to upload the generated images
dataset_obj = omero.model.DatasetI()
v = "ilastik_probabilities_from_dataset_%s" % dataset_id
dataset_obj.setName(omero.rtypes.rstring(v))
v = "ilatisk results probabilities from Dataset:%s" % dataset_id
dataset_obj.setDescription(omero.rtypes.rstring(v))
dataset_obj = conn.getUpdateService().saveAndReturnObject(dataset_obj)

# Prepare ilastik
os.environ["LAZYFLOW_THREADS"] = "2"
os.environ["LAZYFLOW_TOTAL_RAM_MB"] = "2000"
args = ilastik_main.parse_args([])
args.headless = True
args.project = model_file
shell = ilastik_main.main(args)

images = itertools.islice(images, 2)
for image in images:
    filename, file_extension = os.path.splitext(image.getName())
    input_data = load_numpy_array(image)

    # run ilastik headless
    print('running ilastik using %s and %s' % (model_file, image.getName()))
    role_data_dict = OrderedDict(
    [
        (
            "Raw Data",
            [
                PreloadedArrayDatasetInfo(preloaded_array=input_data)
            ],
        )
    ])

    predictions = shell.workflow.batchProcessingApplet.run_export(role_data_dict, export_to_array=True)
    # Save the probabilities file to the image
    print("Saving Probabilities as an Image in OMERO")
    name = filename + "_Probabilities"
    desc = "ilastik probabilities from Image:%s" % image.getId()
    for data in predictions:
        # Re-organise array from tzyxc to zctyx order expected by OMERO
        data = data.swapaxes(0, 1).swapaxes(3, 4).swapaxes(2, 3).swapaxes(1, 2)
        conn.createImageFromNumpySeq(plane_gen(), name,
                                     data.shape[0], data.shape[1],
                                     data.shape[2], description=desc,
                                     dataset=dataset_obj)

print("done")

z:236 t:1 c:2 y:275 x:271
Image converted
running ilastik headless using /tmp/tmpInvF2D/pixel-class-wednesday.ilp on file B1_C1.tif
Saving Probabilities as an Image in OMERO
done


### Close the connection to the OMERO server

In [10]:
conn.close()

### License
Copyright (C) 2019-2020 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.